In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
# Kameraya erişim sağlamak için VideoCapture objesini oluştur
# 0, bilgisayarın varsayılan kamerasını ifade eder
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Kameraya erişilemiyor.")
    exit()

# Döngü içinde görüntüyü sürekli olarak yakala ve ekrana göster
#while True:
    # Kameradan bir kare al
ret, frame = cap.read()

# Eğer görüntü başarıyla alınmadıysa döngüyü kır
if not ret:
    print("Görüntü alınamıyor.")
    #break
    exit()

# Görüntüyü ekrana yazdır

cv2.imshow('Camera', frame)

print(frame.shape)
# 'q' tuşuna basıldığında döngüden çık
if cv2.waitKey(1) & 0xFF == ord('q'):
    #break
    exit()
# İşlem bitince kamera ve pencereleri serbest bırak
cap.release()
cv2.destroyAllWindows()


In [ ]:
# Converting BGR to RGB
#frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

frame_red = frame[:, :, 2]
frame_green = frame[:, :, 1]
frame_blue = frame[:, :, 0]
frame_rgb = np.dstack((frame_red, frame_green, frame_blue))

plt.imshow(frame_rgb)
plt.title("Frame")
plt.show()

In [ ]:
def sliding_window(image, step_size, window_size):
    windows = []
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            windows.append([x, y, image[y:y+window_size[1], x:x+window_size[0]]])

    return windows

In [ ]:
windows = sliding_window(frame_rgb, 100, [64, 128])
for x, y, window in windows:
    plt.imshow(window)
    plt.title("window")
    plt.axis("off")
    plt.show()

In [ ]:
def rgb_to_grayscale(image):
    height, width = image.shape[0:2]
    grayscale_image = np.zeros((height, width), dtype=image.dtype)

    for y in range(height):
        for x in range(width):
            red, green, blue = image[y][x]
            gray = int(red * 0.2989 + green * 0.5870 + blue * 0.1140)
            grayscale_image[y][x] = gray
            
    return grayscale_image

In [ ]:
print(grayscale_frame[0 : 8, 0 : 8])

In [ ]:
def HOG(image):
    x, y, pencere = image
    pencere = np.array(pencere)
    grayscale_frame = rgb_to_grayscale(pencere)
    print(grayscale_frame.shape)

    plt.imshow(grayscale_frame)
    plt.axis("off")
    plt.show()

    height, width = grayscale_frame.shape

    print(height)
    print(width)
    print(grayscale_frame[0:8, 0:8])

    cells = np.zeros((height // 8, width // 8, 8, 8), dtype=grayscale_frame.dtype)
    cell_histogram = np.zeros((height // 8, width // 8, 1, 9), dtype=np.float16)

    for y in range(0, height, 8):
        for x in range(0, width, 8):
            cells[y // 8][x // 8] = grayscale_frame[y : y+8, x : x+8]

    print(cells.shape)
#    print(cells[0][0])
    print("AAAAAAAAAAAAAAAAAaaaa")
    i = 0; j = 0
    histogram_bin = 20
    
    for row in cells:
#        print("row_shape", row.shape)
        for cell in row:
            height_cell, width_cell = cell.shape
#            if i==0:
#                print("cell_shape", cell.shape)
#                print(cell)
#                print("height_cell", height_cell)
#                print("width_cell", width_cell)
            for y in range(0, height_cell, 1):
                for x in range(0, width_cell, 1):
                    if x == 0:
                        gradient_magnitude_x = np.subtract(cell[y][x + 1], cell[y][x], dtype=np.int32)
                    elif x == (width_cell - 1):
                        gradient_magnitude_x = np.subtract(cell[y][x], cell[y][x - 1], dtype=np.int32)
                    else:
                        gradient_magnitude_x = np.subtract(cell[y][x + 1], cell[y][x - 1], dtype=np.int32)

                    if y == 0:
                        gradient_magnitude_y = np.subtract(cell[y + 1][x], cell[y][x], dtype=np.int32)
                    elif y == (height_cell - 1):
                        gradient_magnitude_y = np.subtract(cell[y][x], cell[y - 1][x], dtype=np.int32)
                    else:
                        gradient_magnitude_y = np.subtract(cell[y + 1][x], cell[y - 1][x], dtype=np.int32)

#                    print("gradient_magnitude_x", gradient_magnitude_x)
#                    print("gradient_magnitude_y", gradient_magnitude_y)
                    gradient_magnitude_total = math.sqrt((gradient_magnitude_x ** 2) + (gradient_magnitude_y ** 2))
#                    print("gradient_magnitude_total", gradient_magnitude_total)
                    
                    if gradient_magnitude_x == 0:
                        gradient_angle = 90
                    else:
                        # atan fonksiyonu -pi/2 ile pi/2 aralığında değer verir
                        # gradient_angle = math.atan(gradient_magnitude_y / gradient_magnitude_x)
                        # atan2 fonksiyonu -pi ile pi aralığında değer verir.
                        # abs kullanma nedenimiz çıkan sonucu -'den kurtarıp 0-180 aralığına almak.
                        gradient_angle = abs(math.degrees(math.atan2(gradient_magnitude_y, gradient_magnitude_x)))
#                    print("gradient_angle", gradient_angle)

                    if gradient_angle % histogram_bin == 0:
                        if gradient_angle == 180:
                            gradient_angle = 0
#                        print("cell_histogram_before", cell_histogram[i][j][0][int(gradient_angle / histogram_bin)])    
                        cell_histogram[i][j][0][int(gradient_angle / histogram_bin)] += gradient_magnitude_total
#                        print("cell_histogram", cell_histogram[i][j][0][int(gradient_angle / histogram_bin)])
                    else:
                        remainder = gradient_angle / histogram_bin
                        remainder_float = remainder - int(remainder)
#                        print("remainder", remainder)
#                        print("remainder_float", remainder_float)
#                        print("cell_histogram_before",  cell_histogram[i][j][0][int(remainder)])
                        cell_histogram[i][j][0][int(remainder)] += gradient_magnitude_total * (1 - remainder_float)
#                        print("cell_histogram",  cell_histogram[i][j][0][int(remainder)])
                        if int(remainder) == (cell_histogram[i][j][0].size - 1):
#                            print("cell_histogram_before_for_180", cell_histogram[i][j][0][0])
                            cell_histogram[i][j][0][0] += gradient_magnitude_total * remainder_float
#                            print("cell_histogram_for_180", cell_histogram[i][j][0][0])
                        else:
#                            print("cell_histogram_other_before", cell_histogram[i][j][0][int(remainder) + 1])
                            cell_histogram[i][j][0][int(remainder) + 1] += gradient_magnitude_total * remainder_float
#                            print("cell_histogram_other", cell_histogram[i][j][0][int(remainder) + 1])

            j += 1
#            print("---"*10)
        j = 0    
        i += 1

    HOG_vector = np.zeros((1, 3780), dtype=np.float16)
    index = 0
    print(cell_histogram.shape)
#    print(cell_histogram[0][0])
#    print(cell_histogram[0][1])
    for y in range(cell_histogram.shape[0] - 1):
        for x in range(cell_histogram.shape[1] - 1):
            block_vector = np.concatenate((cell_histogram[y][x], cell_histogram[y][x+1], cell_histogram[y+1][x], cell_histogram[y+1][x+1]), axis = 1)
#            print(block_vector.shape)
#            print(block_vector)
            L2_norm_squared = 0
            for r in block_vector[0]:
                L2_norm_squared += (r ** 2)

#            print("L2_form", L2_norm_squared)
            little_value = 0.0001
            little_value_squared = little_value ** 2
#            print("little_value_squared", little_value_squared)
            v_norm = block_vector / (math.sqrt(L2_norm_squared + little_value_squared))
#            print("v_norm", v_norm)
            
            HOG_vector[0, index:index + v_norm.size] = v_norm.flatten()

            index += v_norm.size

#    visualize_hog(hog_vector=HOG_vector, image_shape=(128, 64))
    return HOG_vector

In [ ]:
print(windows[9])
HOG_original = HOG(windows[9])

In [ ]:
def visualize_hog(hog_vector, image_shape=(128, 64), cell_size=8, bin_count=9, scale_factor=5):
    # Görüntü şekline ve hücre boyutuna göre grid boyutlarını belirleyin
    n_cells_y, n_cells_x = image_shape[0] // cell_size - 1, image_shape[1] // cell_size - 1
    angle_unit = 180 / bin_count  # 0-180 aralığında bins bölmek için

    # HOG vektörünü hücre histogramlarına dönüştür
    hog_vector = hog_vector.flatten().reshape((n_cells_y, n_cells_x, bin_count * 4))

    # Görselleştirme için boş bir görüntü oluştur
    hog_image = np.zeros((image_shape[0], image_shape[1]), dtype=np.float32)

    for y in range(n_cells_y):
        for x in range(n_cells_x):
            # Her hücredeki bins bilgilerini al
            cell_histogram = hog_vector[y, x]
            for bin_idx in range(bin_count):
                # Her bins’te dört hücre var, bu yüzden toplam al
                magnitude = sum(cell_histogram[bin_idx::bin_count])
                angle = bin_idx * angle_unit
                angle_radian = np.deg2rad(angle)

                # Çizgi uzunluğunu ve yönünü belirleyin (ölçek faktörü ekleyin)
                center_x = x * cell_size + cell_size // 2
                center_y = y * cell_size + cell_size // 2
                dx = int(magnitude * scale_factor * np.cos(angle_radian))
                dy = int(magnitude * scale_factor * np.sin(angle_radian))

                # Çizgiyi HOG görseline ekleyin
                cv2.line(hog_image, 
                         (center_x - dx, center_y - dy), 
                         (center_x + dx, center_y + dy), 
                         color=(255,), thickness=1)

    plt.figure(figsize=(10, 5))
    plt.imshow(hog_image, cmap='gray')
    plt.title("HOG Görselleştirmesi")
    plt.show()


In [ ]:
import cv2 as cv
image = cv.imread('human_face.jpg', cv.IMREAD_COLOR)
#image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

#image = rgb_to_grayscale(image)
image = image[10:270, 210:395]
image = cv2.resize(image, (64, 128))
deneme = list([0, 0, image])
print(image.shape)
plt.imshow(image)
plt.show()

HOG_deneme = HOG(deneme)

In [ ]:
frame_deneme = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
frame_deneme = frame_deneme[0:400, 170:430]
frame_deneme = cv.resize(frame_deneme, (64, 128))

plt.imshow(frame_deneme)
plt.show()

frame_deneme2 = list([0,0, frame_deneme])
HOG_original = HOG(frame_deneme2)
HOG_deneme = HOG(deneme)

In [ ]:
distance = np.linalg.norm(HOG_deneme - HOG_original)
print(distance)

In [ ]:
HOG_deneme2 = HOG(deneme)

In [ ]:
distance = np.linalg.norm(HOG_deneme - HOG_deneme2)
print(distance)

In [ ]:
import cv2 as cv
not_face = cv.imread('./test/youtube-83_jpg.rf.15876a9e5ccf00bc202ff3dcebf64c9b.jpg', cv.IMREAD_COLOR)
#not_face = cv.cvtColor(not_face, cv.COLOR_BGR2RGB)

not_face = not_face[0:128, 0:64]
not_face = cv2.resize(not_face, (64, 128))
not_face_deneme = list([0, 0, not_face])
print(not_face.shape)
plt.imshow(not_face)
plt.show()

HOG_not_face = HOG(not_face_deneme)

distance = np.linalg.norm(HOG_not_face - HOG_deneme)
print(distance)